In [1]:
%load_ext autoreload
%autoreload 2
from benchmark.preprocess import *
from benchmark.constants import *
from benchmark.db_import import *
from benchmark.ground_truth import *
from benchmark.construct_threatrace_graph import *
from benchmark.threatrace import *
import logging 
from torch_geometric import seed_everything
import os
import configparser
logging.basicConfig(
    format='%(asctime)s - %(levelname)s - %(message)s',
    level=logging.INFO,
    filename='app.log',
    #filemode='w'
    filemode='a',
)
logging.info('This will get logged to a file')

2023-07-07 09:31:26


In [2]:
project_dir = os.getcwd()
print("Current project directory:", project_dir)

Current project directory: /Users/robinbuchta/Documents/GitHub/gnn_testenv/notebooks


In [3]:
# Read config file
config_dir = "../configs"
config_file_record_03 = config_dir + '/eng3/cadets/cadets_03_record.ini'
config_rec3 = configparser.ConfigParser()
config_rec3.read(config_file_record_03)


['../configs/eng3/cadets/cadets_03_record.ini']

In [4]:
# read tar file and extract to a folder + preprocess
process_darpa_data(config_rec3)

In [5]:
#setup and import data to mariaDB 
import_nodes_and_edges(config_rec3)

In [6]:
# create train and test data
mypygdata = MyPyGDataset(config_rec3, own_timestamp_percent=40, extension="40percentTrain") # 4min
data_train = mypygdata[0][0]
data_test = mypygdata[0][1]
print(data_train)
print(data_test)

Data(x=[244917, 56], edge_index=[2, 2900550], y=[244917], train_mask=[244917], test_mask=[244917])
Data(x=[574338, 56], edge_index=[2, 7375768], y=[574338], train_mask=[574338], test_mask=[574338])


In [7]:
yaml_data = read_gt_yaml(config_rec3)

In [8]:
date, start_ts, end_ts = extract_time_information(yaml_data)

In [9]:
# create ground truth
gt_file1 = config_rec3['Files']['ground_truth'] # -> special case for record3, because here 2 attacks are given 
gt_file2 = config_rec3['Files']['ground_truth1'] # -> special case for record3, because here 2 attacks are given 
gts = [gt_file1, gt_file2]
gt = get_atk_nodes_for_evaluation(config_rec3, multi=True, gts=gts)

In [10]:
# create threatrace GNN anomaly detection Pipeline
ttp = ThreaTracePipeline(config_rec3, data_train, data_test)

In [11]:
#ttp.delete_old_models()
ttp.pretraining()
ttp.multi_model_training()

1 16.098896026611328 0.03909487703997681 2023-07-07 09:34:56
2 14.669391632080078 0.041781501488259286 2023-07-07 09:34:57
3 19.017498016357422 0.13493550876419358 2023-07-07 09:34:58
4 19.12422752380371 0.13756088797429333 2023-07-07 09:35:00
5 18.932720184326172 0.1463107910026662 2023-07-07 09:35:01
6 26.15119743347168 0.1549790337134621 2023-07-07 09:35:02
7 15.833759307861328 0.15643667038221112 2023-07-07 09:35:03
8 33.271461486816406 0.15728593768501165 2023-07-07 09:35:04
9 17.07430076599121 0.1585965857821221 2023-07-07 09:35:06
10 13.842123031616211 0.1620630662632647 2023-07-07 09:35:07
11 9.741694450378418 0.16131179134155652 2023-07-07 09:35:08
12 22.50615692138672 0.18424200851717112 2023-07-07 09:35:09
13 17.80123519897461 0.20311370790921007 2023-07-07 09:35:10
14 24.613996505737305 0.20334643981430445 2023-07-07 09:35:11
15 16.032833099365234 0.20425286933940887 2023-07-07 09:35:12
16 17.58094596862793 0.20936888823560634 2023-07-07 09:35:13
17 9.016636848449707 0.2107

In [12]:
print("##############################################################################################################")
print("own evaluation")
print("##############################################################################################################")
ttp.reinit_test_data() 
ttp.test_model_performance() 


##############################################################################################################
own evaluation
##############################################################################################################
Loop_num: 0  Accuracy:0.1404  true_classified:80628  false_classified:493710
Loop_num: 1  Accuracy:0.7654  true_classified:439626  false_classified:134712
Loop_num: 2  Accuracy:0.0604  true_classified:34674  false_classified:539664
Loop_num: 3  Accuracy:0.0389  true_classified:22353  false_classified:551985
Loop_num: 4  Accuracy:0.0451  true_classified:25882  false_classified:548456
Loop_num: 5  Accuracy:0.0772  true_classified:44337  false_classified:530001
Loop_num: 6  Accuracy:0.0712  true_classified:40887  false_classified:533451
Loop_num: 7  Accuracy:0.1761  true_classified:101134  false_classified:473204
Loop_num: 8  Accuracy:0.0258  true_classified:14802  false_classified:559536
Loop_num: 9  Accuracy:0.0624  true_classified:35866  false_classifie

In [13]:
print("##############################################################################################################")
print("2 HOP")
hits = ttp.evaluation(gt)

##############################################################################################################
2 HOP
{'tn': 574338}
start
{'tn': 574338}
set every node in gt to fn
{'fn': 25049, 'tn': 549289}
gt
{'fn': 25049, 'tn': 549289}


100%|██████████| 68/68 [00:01<00:00, 61.80it/s]


{'fn': 20819, 'tn': 549289, 'tp': 4230}
Intersection Counter: 61
574338
4230 0 549289 20819
TP: 4230, FP: 0, TN: 549289, FN: 20819
Precision:  0.9999999999999764
Recall:  0.168869016727214
F-Score:  0.2889442945208568
Accuracy:  0.9637513102040959


In [14]:
x = ttp.get_detection_insights()

Get Detection Insights
Unique Count of data_flow.y:  (tensor([0, 1, 2, 3, 4, 5]), tensor([133448,  43080, 374521,  12657,  10510,    122]))
Unique Count of pred: (tensor([False,  True]), tensor([574270,     68]))
Unique Count of filtered_tensor: (tensor([0, 1, 2, 3, 4, 5]), tensor([ 1, 18, 15, 26,  1,  7]))
